In [1]:
import os
# save using vs code
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} org.apache.spark:spark-avro_2.12:3.1.3  pyspark-shell'


In [2]:
import findspark
findspark.init()

In [3]:
# PySpark Doesn't support schema registry

In [4]:
import pyspark


from pyspark.sql import SparkSession
# spark groupBy has default setting for spark.sql.shuffle.partitions as 200
# we set to  4, should NOT be done in production 
spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaCandles").getOrCreate()

Exception: Java gateway process exited before sending its port number

In [ ]:
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "hadoop-vm:9092")\
  .option("subscribe", "stock-orders")\
  .option("group.id", "stock-ticks-group-gks")\
  .load()

In [ ]:
from pyspark.sql.avro.functions import from_avro, to_avro


ORDER_SCHEMA_STR = """
{
   "namespace": "com.example",
   "name": "Order",
   "type": "record",
   "fields" : [
     {
       "name" : "symbol",
       "type" : "string"
     },
     {
       "name" : "customer_id",
       "type" : "string"
     },
     {
       "name" : "type",
       "type" : "string"
     },
     {
       "name" : "quantity",
       "type" : "int"
     },
       {
       "name" : "price",
       "type" : "double"
     },
     {
       "name" : "order_type",
       "type" : "string"
     },
     
     {
       "name" : "timestamp",
       "type" : "long"
     }
   ]
}
"""

In [ ]:

# echoOnconsole = ticksDf\
#                 .writeStream\
#                 .outputMode("update")\
#                 .format("console")\
#                 .option("truncate", False)\
#                 .start() # start the query. spark will subscribe for data
